In [1]:
from river import datasets
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from river import drift
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression

In [2]:
# Initialize Dataset
dataset = datasets.synth.Agrawal(
     classification_function=5,
     seed=42
)

# Load the active model pickle file
active_model = pickle.load(open('../saved_models/classification/active_model.pkl', 'rb'))

# Initialie the ADWIN drift detector
drift_detector = drift.ADWIN(delta=1)
warning_detector = drift.ADWIN(delta=1.5)
drifts = []

# Initialise metric
metric = metrics.Accuracy()
print_warning = True
feature_buffer = list()
label_buffer = list()

for i,data in enumerate(dataset.take(420)):
    # data[0] - features, data[1] - label
    y_pred = active_model.predict(np.reshape(list(data[0].values()), (-1, 1)).T)
    metric = metric.update(data[1],y_pred[0])
    correctly_classified = y_pred[0] == data[1]
    warning_detector.update(correctly_classified)
    # Updating buffers for training new model
    feature_buffer.append(list(data[0].values()))
    label_buffer.append(data[1])
    # The warning detector indicates after each sample if there is a warning in the data
    if warning_detector.drift_detected and print_warning:
        print(f'Warning detected at index {i}')
        print_warning = False
        drifts.append(i)
        print(f'Training a new model')
        background_model = LogisticRegression()
        background_model.fit(feature_buffer,label_buffer)
    drift_detector.update(correctly_classified)
    # The drift detector indicates after each sample if there is a drift in the data
    if drift_detector.drift_detected:
        print(f"Change detected at index {i}")
        print_warning = True
        print(f"Replacing active model with background model")
        active_model=background_model

Warning detected at index 191
Training a new model
Change detected at index 415
Replacing active model with background model


/Users/sebasmos/anaconda3/envs/book/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
